In [1]:
# Test run of the CpG Net API.
# Author: Jack Duryea

In [47]:
from CpG_Net import CpGNet
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Automatic reloading so we don't have to restart the kernel each time we make a change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
# Some bins with 2 cpgs
bin1 = np.array([[1,0],[0,-1],[-1,1],[0,0]])
bin2 = np.array([[1,0],[1,0],[-1,1],[0,0],[0,1],[1,1],[0,0]])
bin3 = np.array([[-1,1],[0,-1],[-1,1],[0,0]])
bin4 = np.array([[1,-1],[0,1],[-1,1],[1,0]])
bin5 = np.array([[1,1],[0,0],[0,1],[1,1]])
bin6 = np.array([[1,1],[1,1],[0,1],[1,0]])
bin7 = np.array([[0,0],[1,0],[0,1],[1,1]])



# Collection of bins
bins = [bin1,bin2,bin3,bin4, bin5, bin6, bin7]

In [63]:
net = CpGNet(cpgDensity=2)

In [83]:
net.train(bins)

100%|██████████| 3/3 [00:00<00:00, 299.96it/s]

bins: [<CpG_Net.CpGBin instance at 0x122f47518>, <CpG_Net.CpGBin instance at 0x122f47a28>, <CpG_Net.CpGBin instance at 0x122fabef0>, <CpG_Net.CpGBin instance at 0x122e72440>, <CpG_Net.CpGBin instance at 0x122e720e0>, <CpG_Net.CpGBin instance at 0x122e727a0>, <CpG_Net.CpGBin instance at 0x122e725f0>]
complete_bins: [<CpG_Net.CpGBin instance at 0x122f608c0>, <CpG_Net.CpGBin instance at 0x122f60368>, <CpG_Net.CpGBin instance at 0x122f60950>, <CpG_Net.CpGBin instance at 0x122f60bd8>, <CpG_Net.CpGBin instance at 0x122f60710>, <CpG_Net.CpGBin instance at 0x122f60b90>, <CpG_Net.CpGBin instance at 0x122f60ef0>]
masked_bins 3
masked_bins0 {'observed': array([[ 0.,  0.],
       [ 1., -1.],
       [-1.,  1.],
       [ 1.,  1.]]), 'mask': array([[ 2.,  2.],
       [ 2., -1.],
       [-1.,  2.],
       [ 2.,  2.]]), 'truth': array([[ 0.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  1.]])}
masked_bins1 {'observed': array([[ 1., -1.],
       [ 0.,  0.],
       [-1.,  1.],
       [ 1.,  1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              16000     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1000)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 800)               800800    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 800)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               400500    
__________